# json to npy $\leftarrow$ with Full BT data for e and a 

previously, BT (3D depth point) --> 2D color point <br>
now, save the selected 13joints that are common to kinect and mediapipe as numpy arrays

In [1]:
import numpy as np
import json
from utils import *
import glob
import os
from matplotlib import pyplot as plt
import transformation
#import cv2
import k4a
import device as DEVICE
import json

In [2]:
dtype_npy = get_dtypes(joints_list = "COMMON", isAll = False)


=============== [dtype for npy] ===============
[ 0] ('frame', <class 'int'>)
[ 0] ('wrist_L_x', <class 'float'>)
[ 0] ('wrist_L_y', <class 'float'>)
[ 0] ('elbow_L_x', <class 'float'>)
[ 0] ('elbow_L_y', <class 'float'>)
[ 0] ('shoulder_L_x', <class 'float'>)
[ 0] ('shoulder_L_y', <class 'float'>)
[ 0] ('shoulder_R_x', <class 'float'>)
[ 0] ('shoulder_R_y', <class 'float'>)
[ 0] ('elbow_R_x', <class 'float'>)
[ 0] ('elbow_R_y', <class 'float'>)
[ 0] ('wrist_R_x', <class 'float'>)
[ 0] ('wrist_R_y', <class 'float'>)
[ 0] ('nose_x', <class 'float'>)
[ 0] ('nose_y', <class 'float'>)
[ 0] ('ankle_L_x', <class 'float'>)
[ 0] ('ankle_L_y', <class 'float'>)
[ 0] ('knee_L_x', <class 'float'>)
[ 0] ('knee_L_y', <class 'float'>)
[ 0] ('hip_L_x', <class 'float'>)
[ 0] ('hip_L_y', <class 'float'>)
[ 0] ('hip_R_x', <class 'float'>)
[ 0] ('hip_R_y', <class 'float'>)
[ 0] ('knee_R_x', <class 'float'>)
[ 0] ('knee_R_y', <class 'float'>)
[ 0] ('ankle_R_x', <class 'float'>)
[ 0] ('ankle_R_y', <class '

In [3]:
folder_root  = '/home/hyoyeonlee/Work/DATA_BT/'

######################################
# 1. subfolder name for OUTPUTs (npy arrays)
######################################
subname = '20230506_npy_bt_color2DCoordinate_common_joints_full_data/'

######################################
# 2. Get INPUTs (json files)
######################################

folder_json1 = folder_root + 'skeleton/a/*/BBS/BT/*/*/json/'   #ex) skeleton/a/001/BBS/BT/10/4/json/
folder_json2 = folder_root + 'skeleton/a/*/BBS/BT/*/*/*/json/' #ex) skeleton/e/182/BBS/BT/e/3/4/json/
folders_json = glob.glob(folder_json1) + glob.glob(folder_json2)
folders_json.sort()
print(len(folders_json))
######################################
# 3. Number of Subject(people) & data structure (joints info to be extracted)
######################################
nObjects = 2
#dtype_npy = get_dtypes("COMMON",isAll=False)

5880


In [4]:
######################################
# 4. read json files, extract 13 joints for main subject and assist.-->save as .npy
######################################
with DEVICE.Device.open() as device:
    device_config = k4a.DeviceConfiguration(
            color_format = k4a.EImageFormat.COLOR_BGRA32,
            color_resolution = k4a.EColorResolution.RES_720P,
            depth_mode = k4a.EDepthMode.WFOV_2X2BINNED,
            camera_fps=k4a.EFramesPerSecond.FPS_15,
            synchronized_images_only=True,
            depth_delay_off_color_usec=0,
            wired_sync_mode=k4a.EWiredSyncMode.STANDALONE,
            subordinate_delay_off_master_usec=0,
            disable_streaming_indicator=False)
    status  = device.start_cameras(device_config)
    print(f"*********************************{status}")
    if status != k4a.EStatus.SUCCEEDED:
        exit(-1)
    img_calibration = device.get_calibration(
        depth_mode=device_config.depth_mode,
        color_resolution=device_config.color_resolution)
        ########################################################
        
    img_transformation = transformation.Transformation(img_calibration)
    ecal_depthcam = 0
    ecal_colorcam = 1
    nomitted = 0
    for idx_folder in range(len(folders_json)):
        folder_json = folders_json[idx_folder]
        parse = folder_json.split('/')
        info_camDirection = parse[parse.index('skeleton')+1]
        info_subjectID    = parse[parse.index('skeleton')+2]
        info_taskID       = parse[parse.index('json')-2]
        info_score        = parse[parse.index('json')-1]
        
        files_json = glob.glob(folder_json + "*.json")
        files_json.sort()
        nfiles = len(files_json)
        print(nfiles)
        if nfiles == 0 :
            print(f"[EMPTY] No Files found in {folders_json[idx_folder]}")
            continue
        content_npy  = []
        for idx_file in range(nfiles):
            flag,average_z = get_average_z(extract_points(files_json[idx_file]))
            if flag is True:
                print(f"[no non-zero z values] skip this frame {folders_json[idx_folder]}")
                nomitted+=1
                continue
            content_json = json.load(open(files_json[idx_file]))         
            if idx_file==0:
                for i in range(nObjects):
                    ID = i
                    DATA = np.zeros(nfiles,dtype=dtype_npy)
                    content_npy.append({'id':ID,'data':DATA})
            content_npy[0]['data'][idx_file]['frame'] = idx_file+1
            content_npy[1]['data'][idx_file]['frame'] = idx_file+1
            for idx_obj in range(len(content_json['objects'])):
                if idx_obj==2:
                    break
                obj_json = content_json['objects'][idx_obj]
                for joint_common in COMMON_JOINTS:
                    joint_kinect = KINECT_COMMON[joint_common]
                    x = obj_json['keypoints'][joint_kinect][0]*1.0
                    y = obj_json['keypoints'][joint_kinect][1]*1.0
                    z = obj_json['keypoints'][joint_kinect][2]*1.0
                    if z ==0:
                        z = average_z[idx_obj]
                    point_target = img_transformation.pixel_2d_to_pixel_2d((x,y),z,ecal_depthcam,ecal_colorcam)
                    content_npy[idx_obj]['data'][idx_file][joint_common+"_x"] = point_target[0]
                    content_npy[idx_obj]['data'][idx_file][joint_common+"_y"] = point_target[1]
        for idx_obj in range(nObjects):
            Fname = folder_root+subname+info_camDirection+'/'+info_taskID+'/'+info_score+'/'+info_subjectID
            if not os.path.isdir(Fname):
                os.makedirs(Fname)           
            fname = info_camDirection+'_'+info_subjectID+'_'+info_taskID+'_'+info_score+'_'+str(idx_obj)+'.npy'
            np.save(Fname+'/'+fname,content_npy[idx_obj]['data'])
    device.stop_cameras()
    device.close()
    print(f"***********************************************nomitted = {nomitted}")

[2023-05-08 15:01:51.981] [error] [t=13559] /__w/1/s/extern/Azure-Kinect-Sensor-SDK/src/dynlib/dynlib_linux.c (82): dynlib_create(). Failed to load shared object libdepthengine.so.2.0 with error: libdepthengine.so.2.0: cannot open shared object file: No such file or directory
[2023-05-08 15:01:51.981] [error] [t=13559] /__w/1/s/extern/Azure-Kinect-Sensor-SDK/src/deloader/deloader.cpp (75): deloader_init_once(). Failed to Load Depth Engine Plugin (depthengine). Depth functionality will not work
[2023-05-08 15:01:51.981] [error] [t=13559] /__w/1/s/extern/Azure-Kinect-Sensor-SDK/src/deloader/deloader.cpp (76): deloader_init_once(). Make sure the depth engine plugin is in your loaders path
[2023-05-08 15:01:51.981] [error] [t=13559] /__w/1/s/extern/Azure-Kinect-Sensor-SDK/src/deloader/deloader.cpp (113): deloader_depth_engine_create_and_initialize(). Failed to load depth engine plugin
[2023-05-08 15:01:51.981] [error] [t=13559] /__w/1/s/extern/Azure-Kinect-Sensor-SDK/src/dewrapper/dewrappe

1330
148
383
327
1335
243
166
104
160
165
273
678
349
1266
1270
96
314
310
1280
198
149
118
206
266
756
697
127
1289
1302
128
384
289
1317
221
141
132
243
273
681
678
246
1266
1305
191
454
288
1263
204
170
418
352
361
402
693
185
755
912
253
874
147
670
244
297
430
208
412
644
662
176
726
822
197
751
174
588
240
295
279
307
502
506
472
334
769
372
158
850
204
425
236
343
315
270
480
373
483
307
742
362
181
863
166
413
246
261
354
347
572
177
209
199
140
265
203
753
190
234
168
225
421
357
474
182
190
112
147
248
212
747
144
144
183
186
333
379
490
183
233
157
217
166
214
741
179
276
192
159
288
336
458
220
238
238
199
212
141
752
182
214
155
152
218
327
416
376
459
186
735
386
195
610
238
448
202
199
28
240
489
356
439
303
746
342
171
670
188
442
203
205
265
207
412
513
776
212
656
716
184
488
159
619
210
208
350
213
381
514
818
180
769
727
233
511
165
612
221
238
120
221
238
611
723
236
1360
1317
124
411
296
1263
237
257
105
187
234
618
718
238
1301
1339
140
348
268
1294
168
131
61
15

1406
130
335
252
1446
260
155
90
162
199
357
784
301
1345
1425
92
216
305
1313
165
139
56
71
173
376
808
301
1384
1375
112
310
278
1340
182
116
67
166
186
303
718
321
1303
1317
109
263
288
1403
190
140
151
181
192
314
774
302
1303
1436
110
377
291
1313
186
166
103
221
286
649
773
295
1393
1480
135
338
290
1282
160
186
131
336
289
744
778
281
1341
1473
176
413
296
1347
237
245
287
397
311
713
746
176
665
761
167
806
161
868
297
214
291
403
364
769
763
153
644
750
194
749
175
669
336
207
154
315
420
331
537
242
938
378
157
603
148
569
133
212
225
246
357
460
611
170
915
372
198
699
140
626
185
268
207
265
443
84
116
138
127
305
199
840
128
262
148
143
187
246
374
141
82
111
165
249
159
656
111
185
105
162
193
204
378
64
125
89
124
234
199
697
122
197
69
160
212
190
341
121
115
90
105
183
132
688
92
178
63
158
269
309
389
326
465
254
861
334
122
541
148
520
230
227
216
219
336
452
442
211
853
226
223
645
170
531
203
154
207
227
221
492
961
181
686
768
139
471
186
535
196
174
242
192
335
5

[no non-zero z values] skip this frame /home/hyoyeonlee/Work/DATA_BT/skeleton/a/263/BBS/BT/a/7/3/json/
352
435
116
340
241
740
737
240
1350
1380
141
655
285
1466
334
285
206
433
327
483
728
162
695
781
207
712
211
582
354
340
281
318
302
560
738
163
698
713
180
805
181
718
286
341
122
302
367
410
539
187
1070
394
154
635
195
503
232
288
127
275
413
387
507
174
983
366
108
580
144
453
201
263
233
370
357
148
104
140
107
210
131
720
124
175
113
166
189
286
377
131
77
144
84
150
130
636
87
160
140
169
172
346
333
172
78
111
64
156
135
642
72
132
150
177
123
353
357
129
112
133
75
152
95
632
93
152
156
191
209
394
407
548
477
172
792
315
208
727
201
478
228
263
153
382
404
411
473
217
846
385
121
664
162
612
220
289
301
246
394
491
[no non-zero z values] skip this frame /home/hyoyeonlee/Work/DATA_BT/skeleton/a/275/BBS/BT/a/12/2/json/
725
167
653
759
197
830
148
[no non-zero z values] skip this frame /home/hyoyeonlee/Work/DATA_BT/skeleton/a/275/BBS/BT/a/6/2/json/
579
229
290
321
249
388
441

114
137
378
413
578
476
214
728
280
117
609
123
843
182
162
138
310
386
546
526
257
732
288
137
590
141
682
250
265
224
377
291
785
721
169
741
823
158
718
209
775
290
179
238
0
[EMPTY] No Files found in /home/hyoyeonlee/Work/DATA_BT/skeleton/a/376/BBS/BT/a/10/2/json/
341
825
1263
147
768
809
166
655
153
823
285
144
87
365
145
902
661
204
1459
1362
154
384
334
1440
258
136
133
769
109
850
748
251
1354
629
335
198
294
1473
199
150
41
233
198
279
675
365
714
1339
46
326
360
1512
186
106
40
220
156
285
705
295
1347
1291
99
317
279
1441
269
116
80
261
136
357
764
294
1351
1399
72
226
282
1387
150
104
78
119
151
334
746
292
1350
1347
63
253
264
1328
146
92
87
163
158
573
724
200
1342
1344
109
284
271
1364
196
142
76
198
165
622
817
232
1496
1396
112
340
256
1387
173
145
342
270
310
626
764
141
735
771
146
563
123
720
292
170
342
254
388
634
695
159
791
761
138
611
194
683
220
184
175
357
561
405
417
207
906
309
67
538
139
613
381
209
185
168
617
507
488
279
818
361
144
521
153
708
357
225
2

In [ ]:
!k4aviewer

[2023-05-08 12:42:30.393] [critical] [t=8097] /__w/1/s/extern/Azure-Kinect-Sensor-SDK/src/usbcommand/usbcommand.c (366): find_libusb_device(). libusb device(s) are all unavalable. Is the device being used by another application?
[2023-05-08 12:42:30.393] [error] [t=8097] /__w/1/s/extern/Azure-Kinect-Sensor-SDK/src/depth_mcu/depth_mcu.c (68): usb_cmd_create(USB_DEVICE_DEPTH_PROCESSOR, device_index, NULL, &depthmcu->usb_cmd) returned failure in depthmcu_create()
[2023-05-08 12:42:30.393] [error] [t=8097] /__w/1/s/extern/Azure-Kinect-Sensor-SDK/src/sdk/k4a.c (133): depthmcu_create(index, &device->depthmcu) returned failure in k4a_device_open()
